In [43]:
import re

num_lanes = 3
num_goals = 2
num_freecells = 2

In [ ]:
% lanes to free cells
f(G1, G2, [], F2, [L1|L1s], L2, L3, [S_|S], P, P_) :- L1 \= b, \+ member([L1s,L2,L3],P), step_str(L1, 'L1', 'F1', S_), f(G1, G2, [L1], F2, L1s, L2, L3, S, [[L1s,L2,L3]|P], P_).
f(G1, G2, [], F2, L1, [L2|L2s], L3, [S_|S], P, P_) :- L2 \= b, \+ member([L1,L2s,L3],P), step_str(L2, 'L2', 'F1', S_), f(G1, G2, [L2], F2, L1, L2s, L3, S, [[L1,L2s,L3]|P], P_).
f(G1, G2, [], F2, L1, L2, [L3|L3s], [S_|S], P, P_) :- L3 \= b, \+ member([L1,L2,L3s],P), step_str(L3, 'L3', 'F1', S_), f(G1, G2, [L3], F2, L1, L2, L3s, S, [[L1,L2,L3s]|P], P_).
f(G1, G2, F1, [], [L1|L1s], L2, L3, [S_|S], P, P_) :- L1 \= b, \+ member([L1s,L2,L3],P), step_str(L1, 'L1', 'F2', S_), f(G1, G2, F1, [L1], L1s, L2, L3, S, [[L1s,L2,L3]|P], P_).
f(G1, G2, F1, [], L1, [L2|L2s], L3, [S_|S], P, P_) :- L2 \= b, \+ member([L1,L2s,L3],P), step_str(L2, 'L2', 'F2', S_), f(G1, G2, F1, [L2], L1, L2s, L3, S, [[L1,L2s,L3]|P], P_).
f(G1, G2, F1, [], L1, L2, [L3|L3s], [S_|S], P, P_) :- L3 \= b, \+ member([L1,L2,L3s],P), step_str(L3, 'L3', 'F2', S_), f(G1, G2, F1, [L3], L1, L2, L3s, S, [[L1,L2,L3s]|P], P_).


In [83]:
l = 3
f = 1

L = lambda l,i: [f"L{i}",f"[L{l}|L{l}s]"][l==i]
F = lambda f,i: [f"F{i}",f"[]"][f==i]

L_ = lambda l,i: [f"L{i}",f"L{l}s"][l==i]
F_ = lambda f,l,i: [f"F{i}",f"[L{l}]"][f==i]

res0 = f"""
f(G1, G2, {F(f,1)}, {F(f,2)}, {L(l,1)}, {L(l,2)}, {L(l,3)}, [S_|S], P, P_) :- L{l} \= b, \+ member([{L_(l,1)},{L_(l,2)},{L_(l,3)}],P), step_str(L{l}, 'L{l}', 'F{f}', S_), f(G1, G2, {F_(f,l,1)}, {F_(f,l,2)}, {L_(l,1)}, {L_(l,2)}, {L_(l,3)}, S, [[{L_(l,1)},{L_(l,2)},{L_(l,3)}]|P], P_).
""".strip()



In [84]:
res0 == "f(G1, G2, [], F2, L1, L2, [L3|L3s], [S_|S], P, P_) :- L3 \= b, \+ member([L1,L2,L3s],P), step_str(L3, 'L3', 'F1', S_), f(G1, G2, [L3], F2, L1, L2, L3s, S, [[L1,L2,L3s]|P], P_)."

True

In [85]:
print(res0)

f(G1, G2, [], F2, L1, L2, [L3|L3s], [S_|S], P, P_) :- L3 \= b, \+ member([L1,L2,L3s],P), step_str(L3, 'L3', 'F1', S_), f(G1, G2, [L3], F2, L1, L2, L3s, S, [[L1,L2,L3s]|P], P_).


In [86]:
def lanes_to_freecells(num_lanes, num_freecells):
    L = lambda l,i: [f"L{i}",f"[L{l}|L{l}s]"][l==i]
    F = lambda f,i: [f"F{i}",f"[]"][f==i]

    L_ = lambda l,i: [f"L{i}",f"L{l}s"][l==i]
    F_ = lambda f,l,i: [f"F{i}",f"[L{l}]"][f==i]

    res = []
    for la in range(1,num_lanes+1):
        for lb in range(1,num_lanes+1):
            res0 = f"""
            f(G1, G2, 
                {F(f,1)}, {F(f,2)}, 
                {L(l,1)}, {L(l,2)}, {L(l,3)}, 
                [S_|S], P, P_
            ) :- 
                L{l} \= b, 
                \+ member([{L_(l,1)},{L_(l,2)},{L_(l,3)}],P), 
                step_str(L{l}, 'L{l}', 'F{f}', S_), 
                f(G1, G2, 
                    {F_(f,l,1)}, {F_(f,l,2)}, 
                    {L_(l,1)}, {L_(l,2)}, {L_(l,3)}, 
                    S, [[{L_(l,1)},{L_(l,2)},{L_(l,3)}]|P], P_
                ).
            """
            res.append(re.sub("\n            ","\n",res0).lstrip())

    return "\n".join(res)

In [87]:
lanes_to_freecells(num_lanes, num_freecells)

"f(G1, G2, \n    [], F2, \n    L1, L2, [L3|L3s], \n    [S_|S], P, P_\n) :- \n    L3 \\= b, \n    \\+ member([L1,L2,L3s],P), \n    step_str(L3, 'L3', 'F1', S_), \n    f(G1, G2, \n        [L3], F2, \n        L1, L2, L3s, \n        S, [[L1,L2,L3s]|P], P_\n    ).\n\nf(G1, G2, \n    [], F2, \n    L1, L2, [L3|L3s], \n    [S_|S], P, P_\n) :- \n    L3 \\= b, \n    \\+ member([L1,L2,L3s],P), \n    step_str(L3, 'L3', 'F1', S_), \n    f(G1, G2, \n        [L3], F2, \n        L1, L2, L3s, \n        S, [[L1,L2,L3s]|P], P_\n    ).\n\nf(G1, G2, \n    [], F2, \n    L1, L2, [L3|L3s], \n    [S_|S], P, P_\n) :- \n    L3 \\= b, \n    \\+ member([L1,L2,L3s],P), \n    step_str(L3, 'L3', 'F1', S_), \n    f(G1, G2, \n        [L3], F2, \n        L1, L2, L3s, \n        S, [[L1,L2,L3s]|P], P_\n    ).\n\nf(G1, G2, \n    [], F2, \n    L1, L2, [L3|L3s], \n    [S_|S], P, P_\n) :- \n    L3 \\= b, \n    \\+ member([L1,L2,L3s],P), \n    step_str(L3, 'L3', 'F1', S_), \n    f(G1, G2, \n        [L3], F2, \n        L1, L2, 